# CNN vs ViT 모델 성능 비교

이 노트북에서는 CNN과 ViT 모델의 성능을 비교하고 분석합니다.

## 목표
- CNN과 ViT 모델 성능 비교
- 메트릭 비교 (Accuracy, F1-score, Precision, Recall)
- Loss Curve 비교
- Confusion Matrix 비교
- 오분류 샘플 비교 분석


In [ ]:
# 필요한 라이브러리 import
import sys
sys.path.insert(0, '../src')

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import pandas as pd
from PIL import Image

# 프로젝트 모듈 import
from models.model_utils import create_cnn_model, create_vit_model, load_model_from_checkpoint
from data.dataset import ImageDataset
from data.preprocess import get_val_transforms
from training.evaluator import evaluate_model
from training.metrics import plot_confusion_matrix

print("✅ 라이브러리 로드 완료!")


## 설정 및 데이터 준비


In [ ]:
# 경로 설정
DATA_DIR = Path("../data")
TEST_DIR = DATA_DIR / "test"
CHECKPOINT_DIR = Path("../experiments/checkpoints")
RESULTS_DIR = Path("../experiments/results")
LOG_DIR = Path("../experiments/logs")

# 결과 디렉토리 생성
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# 설정
CONFIG = {
    'image_size': 224,
    'batch_size': 32,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'class_names': ['real', 'fake'],
    'class_map': {'real': '0', 'fake': '1'}
}

# 비교할 모델 설정
MODELS_TO_COMPARE = {
    'CNN_ResNet18': {
        'type': 'cnn',
        'model_name': 'resnet18',
        'checkpoint': CHECKPOINT_DIR / 'CNN_resnet18_best.pth'
    },
    'ViT_Base': {
        'type': 'vit',
        'model_name': 'vit_base',
        'checkpoint': CHECKPOINT_DIR / 'ViT_vit_base_best.pth'
    }
}

print(f"디바이스: {CONFIG['device']}")
print(f"비교할 모델: {list(MODELS_TO_COMPARE.keys())}")


In [ ]:
# 테스트 데이터셋 준비
test_transform = get_val_transforms(CONFIG['image_size'])
test_dataset = ImageDataset(TEST_DIR, CONFIG['class_map'], transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=4)

print(f"테스트 데이터: {len(test_dataset)}개")


## 모델 로드 및 평가


In [ ]:
# 각 모델 평가 및 결과 저장
results = {}
histories = {}

for model_display_name, model_config in MODELS_TO_COMPARE.items():
    print(f"\\n{'='*70}")
    print(f"모델 평가: {model_display_name}")
    print(f"{'='*70}")
    
    checkpoint_path = model_config['checkpoint']
    
    if not checkpoint_path.exists():
        print(f"⚠️  체크포인트를 찾을 수 없습니다: {checkpoint_path}")
        continue
    
    # 모델 로드
    model, checkpoint = load_model_from_checkpoint(
        checkpoint_path=str(checkpoint_path),
        model_type=model_config['type'],
        model_name=model_config['model_name'],
        num_classes=2,
        device=CONFIG['device']
    )
    
    # 평가
    test_results = evaluate_model(
        model,
        test_loader,
        CONFIG['device'],
        class_names=CONFIG['class_names']
    )
    
    # 결과 저장
    results[model_display_name] = {
        'metrics': test_results['metrics'],
        'predictions': test_results['predictions'],
        'labels': test_results['labels']
    }
    
    # 히스토리 로드 (있는 경우)
    if 'history' in checkpoint:
        histories[model_display_name] = checkpoint['history']
    
    print(f"✅ {model_display_name} 평가 완료")
    print(f"   Accuracy: {test_results['metrics']['accuracy']:.4f}")
    print(f"   F1 Score: {test_results['metrics']['f1']:.4f}")


## 성능 메트릭 비교


In [ ]:
# 메트릭 비교 테이블 생성
comparison_data = []
for model_name, result in results.items():
    metrics = result['metrics']
    comparison_data.append({
        'Model': model_name,
        'Accuracy': metrics['accuracy'],
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'F1 Score': metrics['f1']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.set_index('Model')

print("\\n" + "=" * 70)
print("📊 모델 성능 비교")
print("=" * 70)
print(comparison_df.round(4))

# 결과 저장
comparison_df.to_csv(RESULTS_DIR / 'model_comparison.csv')
comparison_df.to_json(RESULTS_DIR / 'model_comparison.json', indent=2)
print(f"\\n✅ 비교 결과 저장: {RESULTS_DIR / 'model_comparison.csv'}")


## 메트릭 시각화 비교


In [ ]:
# 메트릭 비교 바 차트
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']
metric_labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

for idx, (metric, label) in enumerate(zip(metrics_to_plot, metric_labels)):
    ax = axes[idx // 2, idx % 2]
    
    model_names = list(results.keys())
    metric_values = [results[name]['metrics'][metric] for name in model_names]
    
    bars = ax.bar(model_names, metric_values, color=['#3498db', '#e74c3c'][:len(model_names)])
    ax.set_title(f'{label} 비교', fontsize=12, fontweight='bold')
    ax.set_ylabel(label)
    ax.set_ylim([0, 1])
    ax.grid(axis='y', alpha=0.3)
    
    # 값 표시
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'model_comparison_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ 메트릭 비교 그래프 저장: {RESULTS_DIR / 'model_comparison_metrics.png'}")


## Loss Curve 비교


In [ ]:
# Loss Curve 비교
if histories:
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # Loss Curve
    for model_name, history in histories.items():
        axes[0].plot(history['train_loss'], label=f'{model_name} Train', linestyle='--', alpha=0.7)
        axes[0].plot(history['val_loss'], label=f'{model_name} Val', linewidth=2)
    
    axes[0].set_title('Loss Curve 비교', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Accuracy Curve
    for model_name, history in histories.items():
        axes[1].plot(history['train_accuracy'], label=f'{model_name} Train', linestyle='--', alpha=0.7)
        axes[1].plot(history['val_accuracy'], label=f'{model_name} Val', linewidth=2)
    
    axes[1].set_title('Accuracy Curve 비교', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'model_comparison_curves.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Loss Curve 비교 그래프 저장: {RESULTS_DIR / 'model_comparison_curves.png'}")
else:
    print("⚠️  히스토리 데이터를 찾을 수 없습니다.")


## Confusion Matrix 비교


In [ ]:
# Confusion Matrix 비교
num_models = len(results)
fig, axes = plt.subplots(1, num_models, figsize=(6*num_models, 5))

if num_models == 1:
    axes = [axes]

for idx, (model_name, result) in enumerate(results.items()):
    cm = result['metrics']['confusion_matrix']
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=CONFIG['class_names'],
                yticklabels=CONFIG['class_names'],
                ax=axes[idx])
    axes[idx].set_title(f'{model_name}\\nConfusion Matrix', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'model_comparison_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Confusion Matrix 비교 저장: {RESULTS_DIR / 'model_comparison_confusion_matrix.png'}")


## 오분류 샘플 비교 분석


In [ ]:
# 각 모델의 오분류 샘플 찾기
misclassified_by_model = {}

for model_display_name, result in results.items():
    predictions = np.array(result['predictions'])
    labels = np.array(result['labels'])
    
    incorrect_indices = np.where(predictions != labels)[0]
    misclassified_by_model[model_display_name] = incorrect_indices
    
    print(f"{model_display_name}: {len(incorrect_indices)}개 오분류 ({len(incorrect_indices)/len(labels)*100:.2f}%)")

# 공통 오분류 샘플 찾기
if len(misclassified_by_model) >= 2:
    model_names = list(misclassified_by_model.keys())
    common_misclassified = set(misclassified_by_model[model_names[0]])
    for model_name in model_names[1:]:
        common_misclassified = common_misclassified.intersection(set(misclassified_by_model[model_name]))
    
    print(f"\\n공통 오분류 샘플: {len(common_misclassified)}개")
    
    # 각 모델만 틀린 샘플
    for model_name in model_names:
        model_only = set(misclassified_by_model[model_name]) - common_misclassified
        print(f"{model_name}만 틀린 샘플: {len(model_only)}개")


## 종합 분석 및 결론


In [ ]:
print("=" * 70)
print("📊 모델 비교 종합 분석")
print("=" * 70)

# 최고 성능 모델 찾기
best_model = comparison_df['F1 Score'].idxmax()
best_accuracy = comparison_df.loc[best_model, 'Accuracy']
best_f1 = comparison_df.loc[best_model, 'F1 Score']

print(f"\\n🏆 최고 성능 모델: {best_model}")
print(f"   Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"   F1 Score: {best_f1:.4f}")

print(f"\\n📈 모델별 특징:")
for model_name in results.keys():
    metrics = results[model_name]['metrics']
    print(f"\\n{model_name}:")
    print(f"  - Accuracy:  {metrics['accuracy']:.4f} ({metrics['accuracy']*100:.2f}%)")
    print(f"  - Precision: {metrics['precision']:.4f}")
    print(f"  - Recall:    {metrics['recall']:.4f}")
    print(f"  - F1 Score:  {metrics['f1']:.4f}")

print(f"\\n💡 결론:")
if 'CNN' in best_model and 'ViT' in comparison_df.index:
    cnn_f1 = comparison_df.loc[comparison_df.index.str.contains('CNN')[0], 'F1 Score'] if any(comparison_df.index.str.contains('CNN')) else None
    vit_f1 = comparison_df.loc[comparison_df.index.str.contains('ViT')[0], 'F1 Score'] if any(comparison_df.index.str.contains('ViT')) else None
    if cnn_f1 is not None and vit_f1 is not None:
        if cnn_f1 > vit_f1:
            print("  - CNN 모델이 이 데이터셋에서 더 우수한 성능을 보입니다.")
        elif vit_f1 > cnn_f1:
            print("  - ViT 모델이 이 데이터셋에서 더 우수한 성능을 보입니다.")
        else:
            print("  - 두 모델의 성능이 비슷합니다.")

print(f"\\n📁 저장된 파일:")
print(f"  - 비교 테이블: {RESULTS_DIR / 'model_comparison.csv'}")
print(f"  - 메트릭 비교: {RESULTS_DIR / 'model_comparison_metrics.png'}")
print(f"  - Loss Curve 비교: {RESULTS_DIR / 'model_comparison_curves.png'}")
print(f"  - Confusion Matrix 비교: {RESULTS_DIR / 'model_comparison_confusion_matrix.png'}")
print("=" * 70)
